In [1]:
import grpc

from minecraft_pb2 import *
import minecraft_pb2_grpc

import matplotlib.pyplot as plt
import numpy as np
import math

import torch
import diffusers
from tqdm import tqdm
from torchvision import transforms

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from diffusers import UNet2DModel, DDPMScheduler

c:\Mine\Master\MyNca\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
channel = grpc.insecure_channel('localhost:5001')
client = minecraft_pb2_grpc.MinecraftServiceStub(channel)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
blockList = [
    REDSTONE_TORCH, REDSTONE_WIRE, UNPOWERED_REPEATER, UNPOWERED_COMPARATOR, PISTON, COBBLESTONE, AIR, STICKY_PISTON, SLIME,
]

orientationList = [
    NORTH, WEST, SOUTH, EAST, UP, DOWN
]

channels = len(blockList) + len(orientationList)

In [4]:
#Define model

imageSize = 32

timesteps = 200 
scheduler = DDPMScheduler(
    num_train_timesteps=timesteps,
    beta_start=0.0001,
    beta_end=0.02,
    beta_schedule="squaredcos_cap_v2", 
)

model = UNet2DModel(
    sample_size=imageSize,
    in_channels=channels, # random image, conditioning
    out_channels=channels, # Block, orientation
    layers_per_block=2,
    block_out_channels=(64, 128, 256, 512),
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [5]:
#Temp remove cell

offset = {'x': 10, 'y': 4, 'z': 5}
client.fillCube(FillCubeRequest(
    cube=Cube(
        min=Point(x=offset['x'], y=offset['y'], z=offset['z']),
        max=Point(x=offset['x'] + imageSize, y=offset['y'], z=offset['z'] + imageSize)
    ),
    type=AIR
))

In [6]:
def InferNTimes(n):
    model.eval()
    with torch.no_grad():
        # Start from pure noise.
        x = torch.randn((n, channels, imageSize, imageSize)).to(device)

        for t in scheduler.timesteps:
            t_tensor = torch.tensor([t], device=device).long()
            modelOutput = model(x, t_tensor)
            predNoise = modelOutput.sample

            stepOut = scheduler.step(predNoise, t, x)
            x = stepOut.prev_sample

    return x

In [7]:
# Infer model

model.eval()
with torch.no_grad():
    # Start from pure noise.
    x = torch.randn((1, channels, imageSize, imageSize)).to(device)

    for t in scheduler.timesteps:
        t_tensor = torch.tensor([t], device=device).long()
        modelOutput = model(x, t_tensor)
        predNoise = modelOutput.sample

        stepOut = scheduler.step(predNoise, t, x)
        x = stepOut.prev_sample

    sampledImage = x.cpu()

sampledImage.shape

torch.Size([1, 15, 32, 32])

In [8]:
def TensorToBlocks(input, offset):
    blockChannels = input[:, :-len(orientationList), :, :]
    orientationChannels = input[:, -len(orientationList):, :, :]
    
    blockIds = blockChannels.argmax(dim=1).squeeze()
    dirs = orientationChannels.argmax(dim=1).squeeze()

    blocks = []
    for i in range(blockIds.shape[0]):
        for j in range(blockIds.shape[1]):
            blocks.append(
                Block(
                    position=Point(x=i + offset['x'], y=offset['y'], z=j + offset['z']), 
                    type=blockList[blockIds[i, j]], 
                    orientation=orientationList[dirs[i, j]]))
    return Blocks(blocks=blocks)

In [9]:
outblocks = TensorToBlocks(sampledImage, offset)

In [10]:
client.spawnBlocks(outblocks)

In [11]:
nModels = InferNTimes(9)

In [12]:
def TensorsToBlocks(tensors, offset, stride):
    BlocksList = []
    rows = int(math.sqrt(len(tensors)))
    for i in range(len(tensors)):
        buildOffset = offset.copy()
        buildOffset['y'] = 4
        buildOffset['x'] = buildOffset['x'] + i % rows * (stride + imageSize)
        buildOffset['z'] = buildOffset['z'] + int(i / rows) * (stride + imageSize)
        BlocksList.append(TensorToBlocks(tensors[i:i+1], buildOffset))
    return BlocksList # could collect all blocks within a single Blocks(), which would mean the blocks can all be communicated to the server in a single message.

In [13]:
nMcModels = TensorsToBlocks(nModels, offset, 5)

In [16]:
for mcModel in nMcModels:
    client.spawnBlocks(mcModel)

In [15]:
#Temp remove cell
rows = int(math.sqrt(len(nMcModels)))
stride = 5
for i in range(len(nMcModels)):
    buildOffset = offset.copy()
    buildOffset['y'] = 4
    buildOffset['x'] = buildOffset['x'] + i % rows * (stride + imageSize)
    buildOffset['z'] = buildOffset['z'] + int(i / rows) * (stride + imageSize)

    client.fillCube(FillCubeRequest(
        cube=Cube(
            min=Point(x=buildOffset['x'], y=buildOffset['y'], z=buildOffset['z']),
            max=Point(x=buildOffset['x'] + imageSize, y=buildOffset['y'], z=buildOffset['z'] + imageSize)
        ),
        type=AIR
    ))

In [65]:
# Negative log of cosine similarity between predicted noise and actual noise
def replusionLoss(predNoise, noise):
    cosSim = F.cosine_similarity(predNoise, noise, dim=-1)
    return -torch.log(1 - cosSim + 1e-6).mean()

In [57]:
from torch.utils.data import DataLoader, Dataset

In [58]:
class NegativeSampleDataset(Dataset):
    def __init__(self, negativeSamples):
        self.negativeSamples = negativeSamples

    def __len__(self):
        return len(self.negativeSamples)

    def __getitem__(self, idx):
        return self.negativeSamples[idx]

#dataset = NegativeSampleDataset(negativeSamples)
#dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [76]:
batchSize = 4
dataset = NegativeSampleDataset(nModels)
dataloader = DataLoader(dataset, batch_size=batchSize, shuffle=True)

In [63]:
for batch in dataloader:
    print(batch.shape)
    # How many batches does this get?
    # does this make sense

torch.Size([4, 15, 32, 32])
torch.Size([4, 15, 32, 32])
torch.Size([1, 15, 32, 32])


In [77]:
# Negative Training
epochs = 101
model.train()  # Set the model to training mode

for epoch in range(epochs):
    for batch in dataloader:

        optimizer.zero_grad()
        t = torch.randint(0, timesteps, (batch.shape[0],), device=device).long()

        # Sample random noise.
        noise = torch.randn_like(batch)
        noisy_image = scheduler.add_noise(batch, noise, t)

        # Predict the noise using the model.
        model_output = model(noisy_image, t)
        pred_noise = model_output.sample

        # Compute the loss 
        loss = replusionLoss(pred_noise, noise)
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.6f}")

Epoch 0: Loss = -0.640103
Epoch 10: Loss = -0.576219
Epoch 20: Loss = -0.629864
Epoch 30: Loss = -0.592610
Epoch 40: Loss = -0.484804
Epoch 50: Loss = -0.543534
Epoch 60: Loss = -0.249322
Epoch 70: Loss = -0.631184
Epoch 80: Loss = -0.659864
Epoch 90: Loss = -0.249465
Epoch 100: Loss = -0.493726
